In [1]:
import cv2
import torch
from PIL import Image
import time
import socket

# Initialize YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\neelr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-2 Python-3.10.0rc1 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [2]:
def process_video(n, time_interval, video_path, server_socket):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Unable to open video file '{video_path}'")
        return
    
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    time_seconds = time_interval * 1.0
    if time_seconds >= total_frames / frame_rate:
        print(f"Warning: Interval {time_interval} seconds exceeds the duration of video file '{video_path}'. Skipping.")
        cap.release()
        return
    
    frame_number = int(frame_rate * time_seconds)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
    ret, frame = cap.read()
    if not ret:
        print(f"Error: Unable to read frame at {time_interval} seconds from video file '{video_path}'")
        cap.release()
        return

    # Convert frame to PIL image
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)

    # Run YOLOv5 inference
    results = model(frame)
    person_results = results.pred[0][results.pred[0][:, 5] == 0]  # 'person' class is labeled as 0 by default
    
    # Determine if persons are detected
    if len(person_results) > 0:
        person_detected = 1
    else:
        person_detected = 0
    
    # Send binary value indicating whether persons are detected or not
    result = str(n) + str(person_detected)
    server_socket.sendall(result.encode())
    print("Sent:", result)
    
    cap.release()

# Example usage
video_streams = [r"E:\4th sem\IOC-IOT\pro works\fan1-neel.mp4",r'E:\4th sem\IOC-IOT\pro works\test.mp4', r"E:\4th sem\IOC-IOT\pro works\fan3-deepak.mp4"]
time_intervals = [5*i for i in range(10)]  # Define the time intervals


In [5]:
# Server details
SERVER_HOST = '11.12.34.65'  # Type ipconfig in cmd and in wifi get IP4 and paste it 
SERVER_PORT = 65432 # Use any port that is free

# Create a server socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
    server_socket.bind((SERVER_HOST, SERVER_PORT))
    server_socket.listen()
    print("Server is listening for incoming connections...")

    # Accept incoming connections
    conn, addr = server_socket.accept()
    print(f"Connected to client at {addr}")

    # Process each time interval for each video stream
    for interval in time_intervals:
        for video_path in video_streams:
            process_video(video_streams.index(video_path),interval, video_path, conn)
            print()  # Add an empty line for separation between time intervals
        time.sleep(5)  # Sleep for 5 seconds before processing the next interval

    conn.close()

Server is listening for incoming connections...
Connected to client at ('172.19.113.216', 38438)
Sent: 00

Sent: 10

Sent: 21

Sent: 01

Sent: 10

Sent: 21

Sent: 01

Sent: 11

Sent: 21

Sent: 01

Sent: 11

Sent: 20



















